In [ ]:
import numpy as np
import anndata as ad
import scanpy as sc
import smfmodel as smm
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module='anndata')
warnings.filterwarnings("ignore", category=UserWarning, module='anndata')

# Below series of cells is run each time you want to generate new transition matrix solution set

In [ ]:
### Generate solutions of transition matrices for the observed proportion sets derived from the HMM predictions ###

## Provide the observed proportions of states ##
# ORDER MATTERS #
# Order is: Both accessible, Promoter only accessible, Neither Accessible, and Enhancer only accessible.
observed_proportions_on = np.array([0.33, 0.19, 0.29, 0.19])
observed_proportions_off = np.array([0.10, 0.04, 0.53, 0.33])

## Define the variance threshold for a successful steady state solution of a transtion matrix ##
variance_threshold = np.array([0.07, 0.07, 0.08, 0.07])

transition_params = {
    "total_matrices": 1000, # Number of solution matrices to find.
    "size": 4, # Dimensions of the transition matrix (n x n).
    "allow_self_transitions": False, # Whether self-transtions can occur
    "constrain_transitions_to_adjacent": True # Whether to constrain transitions to only adjacent nodes.
}

## Solve for the solutions
transition_matrix_solutions_on = smm.mm.generate_transition_matrix_solutions(observed_proportions_on, variance_threshold, "Active Allele", transition_params)
transition_matrix_solutions_off = smm.mm.generate_transition_matrix_solutions(observed_proportions_off, variance_threshold, "Silent Allele", transition_params)


In [ ]:
### Compile transition matrix solutions from each allele into a single AnnData object ###

## The transition names corresponding to the flattened transition matrix
transition_names = ['Both-Both', 'Both-Promoter_only', 'Both-Neither', 'Both-Enhancer_only', 'Promoter_only-Both', 'Promoter_only-Promoter_only', 'Promoter_only-Neither', 'Promoter_only-Enhancer_only', 'Neither-Promoter_only', 'Neither-Both', 'neither-Neither', 'Neither-Enhancer_only', 'Enhancer_only-Both', 'Enhancer_only-Promoter_only', 'Enhancer_only-Neither', 'Enhancer_only-Enhancer_only']

# Load adata for each condition
adata_off = smm.mm.load_transitions_into_adata(transition_matrix_solutions_off, transition_names, "Silent")
adata_on = smm.mm.load_transitions_into_adata(transition_matrix_solutions_on, transition_names, "Active")

## Concatenate AnnData objects and save the corresponding transition matrix solutions in the unstructured portion of the AnnData
adata = ad.concat([adata_off, adata_on])
# Set obs columns to type 'category'
for col in adata.obs.columns:
    adata.obs[col] = adata.obs[col].astype('category')
adata.obs_names_make_unique()
adata.uns['Active_transition_matrices'] = transition_matrix_solutions_on
adata.uns['Silent_transition_matrices'] = transition_matrix_solutions_off
adata.uns['Transition_array_state_map'] = {k: v for v, k in enumerate(transition_names)}


In [ ]:
## Append detailed balance deviations for the active allele solutions ##
adata.uns['Active_allele_detailed_balance_all_deviations'] = []
adata.uns['Active_allele_detailed_balance_mean_deviations'] = []
adata.uns['Active_allele_detailed_balance_sum_deviations'] = []
adata.uns['Active_allele_dissipation'] = []
for T in transition_matrix_solutions_on:
    deviations, sum_deviations, max_deviation, mean_deviation = smm.mm.detailed_balance_deviation(T)
    adata.uns['Active_allele_detailed_balance_all_deviations'].append(deviations)
    adata.uns['Active_allele_detailed_balance_mean_deviations'].append(mean_deviation)
    adata.uns['Active_allele_detailed_balance_sum_deviations'].append(sum_deviations)
    
## Append detailed balance deviations for the silent allele solutions ##
adata.uns['Silent_allele_detailed_balance_all_deviations'] = []
adata.uns['Silent_allele_detailed_balance_mean_deviations'] = []
adata.uns['Silent_allele_detailed_balance_sum_deviations'] = []
adata.uns['Silent_allele_dissipation'] = []
for T in transition_matrix_solutions_off:
    deviations, sum_deviations, max_deviation, mean_deviation = smm.mm.detailed_balance_deviation(T)
    adata.uns['Silent_allele_detailed_balance_all_deviations'].append(deviations)
    adata.uns['Silent_allele_detailed_balance_mean_deviations'].append(mean_deviation)
    adata.uns['Silent_allele_detailed_balance_sum_deviations'].append(sum_deviations)

In [ ]:
## Save the current AnnData object
adata.write("mm_test_no_self_transitions.h5ad.gz", compression="gzip")
adata

# Start here if you already have an AnnData object containing the above analyses

In [ ]:
### Start from here if you already have an AnnData object
import anndata as ad
import scanpy as sc
adata = ad.read_h5ad("mm_test.h5ad.gz")

In [ ]:
## Display the state transtion names ##
adata.uns['Transition_array_state_map'].keys()

In [ ]:
adata.obs['condition'].cat.categories

In [ ]:
## Example 2D plot of the Promoter only -> Both, vs the Both -> Promoter only kinetics ##
# Below is for the Active allele

x_label = 'Both-Promoter_only'
y_label = 'Promoter_only-Both'
condition = 'Active'
params = {'levels': 25,
          'x_lower': -0.5,
          'x_upper': 1.25,
          'y_lower': -0.5,
          'y_upper': 1.5,
          'save': False}

smm.pl.plot_2D_contour(adata, x_label, y_label, condition, params)

In [ ]:
## Example 2D plot of the Promoter only -> Both, vs the Both -> Promoter only kinetics ##
# Below is for the Silent allele

x_label = 'Both-Promoter_only'
y_label = 'Promoter_only-Both'
condition = 'Silent'
params = {'levels': 25,
          'x_lower': -0.5,
          'x_upper': 1.25,
          'y_lower': -0.5,
          'y_upper': 1.5,
          'save': False}

smm.pl.plot_2D_contour(adata, x_label, y_label, condition, params)

In [ ]:
# Calculate PCA
sc.tl.pca(adata)
# Calculate neighborhood graph
sc.pp.neighbors(adata, n_neighbors=15, use_rep='X_pca')
# Caclulate UMAP
sc.tl.umap(adata)
sc.pl.umap(adata, color='condition')

In [ ]:
## Plot expression densities on UMAP ##
sc.tl.embedding_density(adata, groupby='condition')
sc.pl.embedding_density(adata, groupby='condition')

In [ ]:
# Calculate PCA
sc.tl.pca(adata)
sc.pl.pca(adata, show=True, color='condition')

In [ ]:
## Plot expression densities on PCA ##
sc.tl.embedding_density(adata, basis='pca', groupby='condition')
sc.pl.embedding_density(adata, basis='pca', groupby='condition', color_map='viridis')

In [ ]:
params = {'n_bins': 50,
         'window_size': 2,
         'show_bars': True,
         'show_roll': True,
         'color_palette': 'husl',
         'labels': ['Active', 'Silent'],
          'show_mean': True,
          'show_cdf': False,
         'save': False}

a = adata.uns['Active_allele_detailed_balance_mean_deviations']
b = adata.uns['Silent_allele_detailed_balance_mean_deviations']

smm.pl.plot_hist([a, b], params)

In [ ]:
a = adata.uns['Active_allele_detailed_balance_sum_deviations']
b = adata.uns['Silent_allele_detailed_balance_sum_deviations']
smm.pl.plot_hist([a, b], params)